In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline
import flowdyn.mesh           as mesh
import flowdyn.xnum           as xnum
import flowdyn.integration    as tnum
import flowdyn.modelphy.euler as euler
import flowdyn.modeldisc      as modeldisc
#
plt.rcParams['figure.dpi'] = 120
plt.rcParams['savefig.dpi'] = 120
plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above, uses JavaScript

In [ ]:
nx = 200
meshsim  = mesh.unimesh(ncell=nx,  length=10.)

def S(x): # non-symmetric nozzle
    return 1.+(.2*(x<5)+2*(x>5))*(1-np.exp(-.5*(x-5.)**2))

model = euler.nozzle(sectionlaw=S)
# NPR is ptot/p
bcL = { 'type': 'insub',  'ptot': 3., 'rttot': 1. }
bcR = { 'type': 'outsub', 'p': 1. }

monitors = {
    'residual':{ 'name':'L2 residuals', 'frequency': 10 },
    'data_average':{ 'data': 'mach', 'name':'Mach average', 'frequency': 1 }
    }
rhs    = modeldisc.fvm(model, meshsim, xnum.muscl(xnum.vanalbada), bcL=bcL, bcR=bcR)

finit = rhs.fdata_fromprim([  1., 0.5, 1. ]) # rho, u, p

In [ ]:
params = {
    'RK3SSP - global CFL 2.' : (tnum.rk3ssp, 1., ''),
    'RK3SSP - local CFL 2.' : (tnum.rk3ssp, 1., 'dtlocal'),
} 
output = dict()

for name, (integ, cfl, dtdirective) in params.items():
    output[name] = dict()
    print(name, cfl, dtdirective)
    mon = {key: value.copy() for key, value in monitors.items()}
    dir = {'verbose': True, dtdirective: ''}
    solver = integ(meshsim, rhs, monitors=mon)
    fsol = solver.solve(finit, cfl, stop={'maxit': 4000}, directives=dir)
    for imon in mon.values():
        output[name][imon['name']] = imon['output']
    output[name]['perf'] = solver.show_perf()

In [ ]:
# Figure / Plot of final results
fig, ax = plt.subplots(1, len(monitors.keys()), figsize=(10,4))
for iax, lab in zip(ax, mon.values()):
    iax.grid(linestyle='--', color='0.5')
    iax.set_ylabel(lab['name'])
for name in params.keys():
    output[name]['L2 residuals'].semilogplot_it(ax=ax[0])
    output[name]['Mach average'].plot_it(ax=ax[1])

In [ ]:
fsol[-1].plot('mach')